## **Queries SQL para MySQL Server 8.4.0**

1. A base possui diversos valores nulos. Preencha nas colunas onde os valores são nulls com 'NAN'.

In [ ]:
-- Obtendo uma lista com nomes das colunas
SELECT 
    column_name 
FROM 
    INFORMATION_SCHEMA.COLUMNS 
WHERE 
    table_name='netflix';

In [ ]:
-- Substituindo NULL por 'NAN'
-- COALESCE retorna o valor do segundo parâmetro caso o primeiro seja NULL
UPDATE netflix SET cast=COALESCE(cast, 'NAN');
UPDATE netflix SET country=COALESCE(country, 'NAN');
UPDATE netflix SET date_added=COALESCE(date_added, 'NAN');
UPDATE netflix SET director=COALESCE(director, 'NAN');
UPDATE netflix SET duration=COALESCE(duration, 'NAN');
UPDATE netflix SET listed_in=COALESCE(listed_in, 'NAN');
UPDATE netflix SET media_type=COALESCE(media_type, 'NAN');
UPDATE netflix SET rating=COALESCE(rating, 'NAN');
UPDATE netflix SET release_year=COALESCE(release_year, 'NAN');
UPDATE netflix SET show_id=COALESCE(show_id, 'NAN');
UPDATE netflix SET synopsis=COALESCE(synopsis, 'NAN');
UPDATE netflix SET title=COALESCE(title, 'NAN');

2. Normalize a coluna CAST criando uma nova tabela 'cast_table' de modo que tenhamos separadamente, ou seja, uma coluna com o nome do elenco de cada filme. Exemplo:  

Linha: n1 joao, maria, roberto  
coluna:  
id CAST  
n1 joao  
n1 maria  
n1 roberto  

3. Normalize a coluna listed_in criando uma nova tabela 'genre_table' de modo que tenhamos separadamente os gêneros de cada programação. Exemplo:  

Linha: n1 Ação, Aventura, Comédia  
coluna:  
n1 Ação  
n1 Comédia  
n1 Aventura  

4. Normalize a coluna date_added em uma nova base 'date_table' e  construa as seguintes colunas:

**coluna day: DD**

In [ ]:
-- Criando a coluna day do tipo INT
ALTER TABLE netflix
ADD COLUMN day INT;

/* Sendo a data do tipo 'September 25, 2021'
substring_index(date_added,',', 1) obtém o conteúdo até a ocorrência da primeira vírgula (September 25)
(substring_index(substring_index(date_added,',', 1),' ', -1)) retorna o primeiro valor após a 
ocorrência do primeiro espaço em branco (25) */
UPDATE netflix
SET day = (substring_index(
			substring_index(date_added,',', 1),
            ' ', -1))
                WHERE date_added != 'NAN';

**coluna month: MM**

In [ ]:
-- Criando a coluna month do tipo INT
ALTER TABLE netflix
ADD COLUMN month INT;

/* STR_TO_DATE transforma uma string em uma data formatada e MONTH obtém o número do mês da data 
WHERE está sendo usado para ignorar os valores 'NAN'*/
UPDATE netflix
SET month = MONTH(STR_TO_DATE(date_added, '%M %d, %Y'))
                WHERE date_added != 'NAN';

**coluna year: YY**

In [ ]:
ALTER TABLE netflix
ADD COLUMN year INT;

UPDATE netflix
SET year = YEAR(STR_TO_DATE(date_added, '%M %d, %Y'))
                WHERE date_added != 'NAN';

**coluna iso_date_1: YYYY-MM-DD**

In [ ]:
ALTER TABLE netflix
ADD COLUMN iso_date_1 DATE;

UPDATE netflix
SET iso_date_1 = date_format(STR_TO_DATE(date_added, '%M %d, %Y'), '%Y-%m-%d') 
                WHERE date_added != 'NAN';

**coluna iso_date_2: YYYY/MM/DD**

In [ ]:
ALTER TABLE netflix
ADD COLUMN iso_date_2 VARCHAR(20);

UPDATE netflix
SET iso_date_2 = date_format(STR_TO_DATE(date_added, '%M %d, %Y'), "%Y/%m/%d") 
                WHERE date_added != 'NAN';

**coluna iso_date_3: YYMMDD**

In [ ]:
ALTER TABLE netflix
ADD COLUMN iso_date_3 VARCHAR(20);

UPDATE netflix
SET iso_date_3 = date_format(STR_TO_DATE(date_added, '%M %d, %Y'), '%y%m%d') 
                WHERE date_added != 'NAN';

**coluna iso_date_4: YYYYMMDD**

In [ ]:
ALTER TABLE netflix
ADD COLUMN iso_date_4 VARCHAR(20);

UPDATE netflix
SET iso_date_4 = date_format(STR_TO_DATE(date_added, '%M %d, %Y'), '%Y%m%d') 
                WHERE date_added != 'NAN';

5. Normalize a coluna duration e construa uma nova base 'time_table' e faça as seguintes conversões.
- Converta a coluna duration para horas e crie a coluna hours hh. Obs. A média de cada
season TV SHOW é 10 horas, assim também converta para horas.  
- Converta todas as horas para minutos e armazena na coluna minutes mm.  

In [ ]:
-- Criando a tabela 'time_table'
CREATE TABLE projeto.time_table(
		show_id VARCHAR(20),
		hours FLOAT, 
        minutes INT
);

-- Criando a coluna 'hours' na tabela 'netflix'
ALTER TABLE netflix
ADD COLUMN hours FLOAT;

/* Calculando as horas 
- Temporadas
Expressão regular identifica Season ou Seasons e 
o valor inteiro (vindo antes do espaço em branco) é multiplicado por 10
- Minutos 
Divide o valor inteiro (vindo antes do espaço em branco) por 60 */

UPDATE netflix
SET hours = 
    CASE
		WHEN LOWER(duration) REGEXP '.*season.*' THEN (SUBSTRING_INDEX(duration, ' ', 1) * 10)
        ELSE (SUBSTRING_INDEX(duration, ' ', 1) / 60)
    END
WHERE duration != 'NAN';

-- Criando a coluna 'minutes' na tabela 'netflix'
ALTER TABLE netflix
ADD COLUMN minutes INT;

-- Multiplicando o valor de horas por 60 para obter minutos
UPDATE netflix
SET minutes = hours * 60
WHERE hours IS NOT NULL;

-- Inserindo os valores das columas 'show_id', 'hours' e 'minutes' na tabela 'time_table'
INSERT INTO time_table(show_id, hours, minutes)
SELECT show_id, hours, minutes FROM netflix;

-- Removendo as colunas 'hours' e 'minutes' na tabela 'netflix'
ALTER TABLE netflix DROP COLUMN hours;
ALTER TABLE netflix DROP COLUMN minutes;

6. Normalize a coluna country criando uma nova tabela 'country_table' de modo que tenhamos separadamente
uma coluna com o nome do país de cada filme.

7. Qual o filme de duração máxima em minutos?

In [ ]:
/* Obtendo somente o primeiro filme com valor máximo:
- Ordena os valores de duração em minutos em ordem decrescente
- Obtém o primeiro registro */
SELECT 
    netflix.title, netflix.show_id, time_table.minutes
FROM  
    time_table
LEFT JOIN 
    netflix 
ON 
    time_table.show_id = netflix.show_id
WHERE 
    netflix.media_type = 'Movie'
ORDER BY 
    time_table.minutes DESC
LIMIT 
    1;

-- Obtendo todos os filmes com duração igual a máxima

/* Obtendo todos os filmes com duração igual a máxima
- Obtém o valor máximo para todos os filmes na tabela 'time_table'
- Obtém todos os filmes cujo tempo de duração é igual ao valor máximo*/
SELECT 
    netflix.show_id, netflix.title, time_table.minutes
FROM 
    time_table
LEFT JOIN 
    netflix 
ON 
    time_table.show_id = netflix.show_id
WHERE 
    netflix.media_type = 'Movie' and time_table.minutes=(
    -- Obtendo duração máxima dos filmes
    SELECT 
        max(minutes)
    FROM 
        time_table
    LEFT JOIN 
        netflix
    ON 
        time_table.show_id = netflix.show_id
    WHERE 
    netflix.media_type = 'Movie'
);

8. Qual o filme de duração mínima em minutos?

In [ ]:
SELECT
     netflix.show_id, netflix.title, time_table.minutes
FROM
     time_table
LEFT JOIN
     netflix 
ON
     time_table.show_id = netflix.show_id
WHERE
     netflix.media_type = 'Movie' and time_table.minutes IS NOT NULL
ORDER BY
     time_table.minutes
LIMIT
     1;

SELECT
     netflix.show_id, netflix.title, time_table.minutes
FROM
     time_table
LEFT JOIN
     netflix 
ON
     time_table.show_id = netflix.show_id
WHERE
     netflix.media_type = 'Movie' and time_table.minutes=(
    SELECT 
    min(minutes)
    FROM
        time_table
    LEFT JOIN
        netflix
    ON
        time_table.show_id = netflix.show_id
    WHERE
        netflix.media_type = 'Movie');

9. Qual a série de duração máxima em minutos?

In [ ]:
SELECT
    netflix.title, netflix.show_id, time_table.minutes
FROM
    time_table
LEFT JOIN
    netflix 
ON
    time_table.show_id = netflix.show_id
WHERE
    netflix.media_type = 'TV Show'
ORDER BY
    time_table.minutes DESC
LIMIT
    1;

SELECT
    netflix.show_id, netflix.title, time_table.minutes
FROM time_table
LEFT JOIN
    netflix 
ON
    time_table.show_id = netflix.show_id
WHERE
    netflix.media_type = 'TV Show' and time_table.minutes=(SELECT
    max(minutes)
FROM
    time_table
LEFT JOIN
    netflix
ON
    time_table.show_id = netflix.show_id
WHERE
    netflix.media_type = 'TV Show');

10. Qual a série de duração mínima em minutos?

In [ ]:
SELECT
    netflix.show_id, netflix.title, time_table.minutes
FROM
    time_table
LEFT JOIN
    netflix 
ON
    time_table.show_id = netflix.show_id
WHERE
    netflix.media_type = 'TV Show' and time_table.minutes IS NOT NULL
ORDER BY
    time_table.minutes
LIMIT
    1;

SELECT
    netflix.show_id, netflix.title, time_table.minutes
FROM
    time_table
LEFT JOIN
    netflix 
ON
    time_table.show_id = netflix.show_id
WHERE
    netflix.media_type = 'TV Show' and time_table.minutes=(
    SELECT
        min(minutes)
    FROM
        time_table
    LEFT JOIN
        netflix
    ON
        time_table.show_id = netflix.show_id
    WHERE
        netflix.media_type = 'TV Show');

11. Qual a média de tempo de duração dos filmes?

In [ ]:
-- Em minutos
SELECT
    avg(minutes)
FROM
    time_table
LEFT JOIN
    netflix
ON
    time_table.show_id = netflix.show_id
WHERE
    netflix.media_type = 'Movie';

-- Em horas
SELECT
    avg(hours)
FROM
    time_table
LEFT JOIN
    netflix
ON
    time_table.show_id = netflix.show_id
WHERE
    netflix.media_type = 'Movie';

12. Qual a média de tempo de duração das séries?

In [ ]:
-- Em minutos
SELECT
    avg(minutes)
FROM
    time_table
LEFT JOIN
    netflix
ON
    time_table.show_id = netflix.show_id
WHERE
    netflix.media_type = 'Tv Show';

-- Em horas
SELECT
    avg(hours)
FROM
    time_table
LEFT JOIN
    netflix
ON
    time_table.show_id = netflix.show_id
WHERE
    netflix.media_type = 'Tv Show';

13. Qual a lista de filmes o ator Leonardo DiCaprio participa?

In [ ]:
SELECT
    title
FROM
    netflix
WHERE
    cast 
LIKE
    '%Leonardo DiCaprio%'
AND
    media_type='Movie';

14. Quantas vezes o ator Tom Hanks apareceu nas telas do netflix, ou seja, tanto série quanto filmes?

In [ ]:
SELECT 
    count(*)
FROM 
    netflix
WHERE 
    cast LIKE '%Tom Hanks%';

15. Quantas produções séries e filmes brasileiras já foram ao ar no netflix?

In [ ]:
-- Contabilizando produções unicamente brasileiras 
SELECT 
    count(*)
FROM 
    netflix
WHERE 
    country='Brazil';

-- Contabilizando produções que também envolvem outros países além do Brasil
SELECT 
    count(*)
FROM 
    netflix
WHERE 
    country LIKE '%Brazil%';

16. Quantos filmes americanos já foram para o ar no netflix?

In [ ]:
SELECT 
    count(*)
FROM 
    netflix
WHERE 
    country='United States';

SELECT 
    count(*)
FROM 
    netflix
WHERE 
    country LIKE '%United States%';

17. Crie uma nova coluna com o nome last_name_director com uma nova formatação para o nome dos diretores, por exemplo. João Roberto para Roberto, João.

18. Procure a lista de conteúdos que tenha como temática a segunda guerra mundial (WWII)?

In [ ]:
SELECT 
    title 
FROM 
    netflix 
WHERE 
    synopsis LIKE '%WWII%';

19. Conte o número de produções dos países que apresentaram conteúdos no netflix?

In [ ]:
SELECT 
    country, count(country) as numero_producoes 
FROM 
    netflix 
GROUP BY
    country 
ORDER BY 
    country;